# A1367 OC - Extinction Download

## Imports

In [ ]:
# Astropy Collab Imports
from astropy import units as u
from astropy.io import fits
from astropy.table import QTable
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astroquery.ipac.irsa.irsa_dust import IrsaDust

## Load External Data

### Load Image Data

The center of the image *might* not correspond to the NED galaxy used to find the GAIA
sources. Therefore, we will instead get the `CR_PIX` coordinate from one of the processed images
to know what the coordinate we need for extinction.

In [ ]:
# Open the Image and Get the CR_PIX Coordinate
with fits.open('../../Images/ProcessedImages/HST/DrizzledImages/A1367OC-F814W_drc.fits') as hduList:

    # Get the WCS from the FITS Header
    wcs = WCS(hduList[1])

# Get the Coordinate for the Center of the Image
IMG_COORD = SkyCoord(
    wcs.wcs.crval[0],
    wcs.wcs.crval[1],
    unit=(u.deg, u.deg),
    frame=wcs.wcs.radesys.lower()
)

### Get the IRSA Extinction Data 

In [ ]:
# Get the E(B-V) Value for the Image
# Schlafly, E.F. & Finkbeiner, D.P.  2011, ApJ 737, 103 (S and F).
# Steps on the One Line Below
#   Query the Table -- get_query_table
#   Get the E(B-V) Value -- > ['ext SandF mean']
#   Get the Astropy Table Column as an array --> .value
#   Get the Single Value from the Array --> .item()
ebv  = IrsaDust.get_query_table(IMG_COORD, section='ebv')['ext SandF mean'].value.item()
ebv *= u.mag

# Get the Full Extinction Table
table = IrsaDust.get_extinction_table(IMG_COORD)
table = QTable(table)

## Write Data to File

In [ ]:
# Write the E(B-V) data to a File
with open('A1367OC-EBV.txt', 'w') as file:
    file.write(f'{ebv}\n')

In [ ]:
# Write the Extinction Table to a File
table.write('A1367OC-ExtinctionTable.ecsv', overwrite=True)